Code in this notebook is used to plot PACF and ACF plots of the correlation coefficient matrix calculated earlier.
Also methods which fit and evaluate ARIMA models are also present here. Afte rprediction, residual values are 
saved in csv's which will be later fed to LSTM model for drawing non linear tendencies.

In [ ]:
#Cell 1 -Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.stats as stats
import pylab as pl
from pmdarima.arima import ARIMA, auto_arima
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.simplefilter("ignore")

In [ ]:
#Cell 2 - Load correlation coefficient matrix

data_df = pd.read_csv('/Users/gauravthapliyal/Desktop/Project Data/stock_correlation_prediction/Correlation_Matrix.csv')
data_df = data_df.loc[:, ~data_df.columns.str.contains('^Unnamed')]
print(data_df.shape)

num_list = []
for i in range(24):
    num_list.append(str(i))
data_df = data_df[num_list].copy()
data_df = np.transpose(data_df)
print(data_df.shape)
print(data_df.head())

In [ ]:
# Cell 3 - Break down data in 4 equal parts which will be treated as train/dev/test1/test2 values.
#These values will act as the actual values and predicted values will be matched against these.

indices = [20*k for k in range(55875)]
data_df = pd.DataFrame(data_df[indices])

train = []
dev = []
test1 = []
test2 = []

#division of data in 4 parts
for i in range(data_df.shape[1]):
    tmp = data_df[20*i].copy()
    train.append(tmp[:21])
    dev.append(tmp[1:22])
    test1.append(tmp[2:23])
    test2.append(tmp[3:24])
    
train = pd.DataFrame(train)
dev = pd.DataFrame(dev)
test1 = pd.DataFrame(test1)
test2 = pd.DataFrame(test2)

# saving data sets to csv
train.to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaTrain.csv')
dev.to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaDev.csv')
test1.to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaTest1.csv')
test2.to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaTest2.csv')


In [ ]:
#Cell 4 - Plot ACF and PACF plots for the train dat set

train = pd.read_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaTrain.csv')
train = np.transpose(train.loc[:, ~train.columns.str.contains('^Unnamed')])
#Plotting random data from train dataset to get a idea of the distribution. This helps in determining parameters
#for SARIMAX model
for _ in range(100):
    randint = random.randrange(0,55875,1)
    print(randint)
    train[randint].plot()
    plt.show()
    plt.close()
    plot_acf(train[randint].diff()[1:])
    plt.show()
    plt.close()
    plot_pacf(train[randint].diff()[1:])
    plt.show()
    plt.close()
    print('-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-')

In [ ]:
stat = pd.DataFrame()
for i in range(55875):
    df = train[i].describe()
    stat[i] = df

In [ ]:
#Cell 5 - ARIMA Model

#Load data from train, dev, test set

train = pd.read_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaTrain.csv')
dev = pd.read_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaDev.csv')
test1 = pd.read_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaTest1.csv')
test2 = pd.read_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Pre_ARIMA/ArimaTest2.csv')


train = np.transpose(train.loc[:,~train.columns.str.contains('^Unnamed')])
dev = np.transpose(dev.loc[:,~dev.columns.str.contains('^Unnamed')])
test1 = np.transpose(test1.loc[:,~test1.columns.str.contains('^Unnamed')])
test2 = np.transpose(test2.loc[:,~test2.columns.str.contains('^Unnamed')])

datasets = [train, dev, test1, test2]


#Models which we will be using to fit ARIMA model.
#We have taken a seasonal order of 12 
#p, d, q have standard ARIMA model values

model_110 = ARIMA(order=(1,1,0), mle_regression=True, suppress_warnings=True)
model_011 = ARIMA(order=(0,1,1), mle_regression=True, suppress_warnings=True)
model_111 = ARIMA(order=(1,1,1), mle_regression=True, suppress_warnings=True)
model_211 = ARIMA(order=(2,1,1), mle_regression=True, suppress_warnings=True)
model_210 = ARIMA(order=(2,1,0), mle_regression=True, suppress_warnings=True)

train_X = []; train_Y = []
dev_X = []; dev_Y = []
test1_X = []; test1_Y = []
test2_X = []; test2_Y = []

flag = 0

#Iterating thorugh different SARIMAX models for all the data sets
for i in range(55875):
    print(i)
    tmp = []
    c=0
    for s in datasets :
        c+=1
        try:
            model1 = model_110.fit(s[i])
            model = model1
            
            try:
                model2 = model_011.fit(s[i])
                
                #use of Akaike Information Criterion for determining better fitting model
                if model.aic() <= model2.aic() :
                    pass
                else :
                    model = model2
                    
                try :
                    model3 = model_111.fit(s[i])
                    if model.aic() <= model3.aic() :
                        pass
                    else :
                        model = model3
                except :
                    try:
                        model4 = model_211.fit(s[i])
                        
                        if model.aic() <= model4.aic() :
                            pass
                        else:
                            model = model4
                    except:
                        try:
                            model5 = model_210.fit(s[i])
                            
                            if model.aic() <= model5.aic():
                                pass
                            else :
                                model = model5
                        except :
                            pass
                    
            except:
                try:
                    model3 = model_111.fit(s[i])

                    if model.aic() <= model3.aic() :
                        pass
                    else :
                        model = model3
                except :
                    try:
                        model4 = model_211.fit(s[i])
                        
                        if model.aic() <= model4.aic() :
                            pass
                        else:
                            model = model4
                    except:
                        try:
                            model5 = model_210.fit(s[i])
                            
                            if model.aic() <= model5.aic():
                                pass
                            else :
                                model = model5
                        except :
                            pass
                
        except:
            try:
                model2 = model_011.fit(s[i])
                model = model2
            
                try :
                    model3 = model_111.fit(s[i])
                    
                    if model.aic() <= model3.aic():
                        pass
                    else:
                        model = model3
                except :
                    try:
                        model4 = model_211.fit(s[i])
                        
                        if model.aic() <= model4.aic() :
                            pass
                        else:
                            model = model4
                    except:
                        try:
                            model5 = model_210.fit(s[i])
                            
                            if model.aic() <= model5.aic():
                                pass
                            else :
                                model = model5
                        except :
                            pass
            
            except :
                try:
                    model3 = model_111.fit(s[i])
                    model = model3
                except :
                    try:
                        model4 = model_211.fit(s[i])
                        
                        if model.aic() <= model4.aic() :
                            pass
                        else:
                            model = model4
                    except:
                        try:
                            model5 = model_210.fit(s[i])
                            
                            if model.aic() <= model5.aic():
                                pass
                            else :
                                model = model5
                        except :
                            flag = 1
                            print(str(c) + " FATAL ERROR")
                            break
        
        predictions = list(model.predict_in_sample())
        #pad the first time step of predictions with the average of the prediction values
        #so as to match the length of the s[i] data
        predictions = [np.mean(predictions)] + predictions
        
        residual = model.resid()
        tmp.append(np.array(residual))
        
                    
    if flag == 1:
        break
    train_X.append(tmp[0][:20])
    train_Y.append(tmp[0][20])
    dev_X.append(tmp[1][:20])
    dev_Y.append(tmp[1][20])
    test1_X.append(tmp[2][:20])
    test1_Y.append(tmp[2][20])
    test2_X.append(tmp[3][:20])
    test2_Y.append(tmp[3][20])
    
#Save residual values which will be used as input to LSTM model
pd.DataFrame(train_X).to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Post_ARIMA/train_X.csv')
pd.DataFrame(dev_X).to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Post_ARIMA/dev_X.csv')
pd.DataFrame(test1_X).to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Post_ARIMA/test1_X.csv')
pd.DataFrame(test2_X).to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Post_ARIMA/test2_X.csv')
pd.DataFrame(train_Y).to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Post_ARIMA/train_Y.csv')
pd.DataFrame(dev_Y).to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Post_ARIMA/dev_Y.csv')
pd.DataFrame(test1_Y).to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Post_ARIMA/test1_Y.csv')
pd.DataFrame(test2_Y).to_csv('/Users/gauravthapliyal/Desktop/Project Data/train_dev_test/Post_ARIMA//test2_Y.csv')
        